In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/gonzalo/Downloads/ml4cc-general-access_request_pays.json"
os.environ["GS_USER_PROJECT"] = "ml4cc-general"

In [2]:
%%time
from georeader.readers import S2_SAFE_reader

safe_file = "S2B_MSIL1C_20220527T030539_N0400_R075_T49SGV_20220527T051042.SAFE"
s2_safe_folder = S2_SAFE_reader.s2_public_bucket_path(safe_file, check_exists=True)

print(f"File is located at: {s2_safe_folder}")

s2obj = S2_SAFE_reader.s2loader(s2_safe_folder, out_res=10)
s2obj

File is located at: gs://gcp-public-data-sentinel-2/tiles/49/S/GV/S2B_MSIL1C_20220527T030539_N0400_R075_T49SGV_20220527T051042.SAFE
CPU times: user 777 ms, sys: 398 ms, total: 1.17 s
Wall time: 951 ms


 
         Transform: | 10.00, 0.00, 699960.00|
| 0.00,-10.00, 4000020.00|
| 0.00, 0.00, 1.00|
         Shape: (13, 10980, 10980)
         Resolution: (10.0, 10.0)
         Bounds: (699960.0, 3890220.0, 809760.0, 4000020.0)
         CRS: EPSG:32649
         bands: ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12']
         fill_value_default: 0
        

## Metadata files in Level 1C images

There are two metadata files (that I know) in Sentinel-2 Level 1C images, these are saved in the attributes
 * `s2obj.metadata_msi` 
 * `s2obj.metadata_tl`

In [3]:
s2obj.metadata_msi

'gs://gcp-public-data-sentinel-2/tiles/49/S/GV/S2B_MSIL1C_20220527T030539_N0400_R075_T49SGV_20220527T051042.SAFE/MTD_MSIL1C.xml'

In [7]:
s2obj.metadata_tl

'gs://gcp-public-data-sentinel-2/tiles/49/S/GV/S2B_MSIL1C_20220527T030539_N0400_R075_T49SGV_20220527T051042.SAFE/GRANULE/L1C_T49SGV_A027271_20220527T031740/MTD_TL.xml'

### Reflectance to radiance conversion

We want the pixels of our images in [**spectral radiances**](https://en.wikipedia.org/wiki/Radiance) with units $W·sr^{-1}·m^{−2}·nm^{−1}$. Spectral raciances are (*watts per steradian per square meter per nanometer*).

According to this https://gis.stackexchange.com/questions/285996/convert-sentinel-2-1c-product-from-reflectance-to-radiance the formula to convert digital numbers (DN) in ToA images is:

toaBandX = (pixelValueBandX + radioAddOffsetBandX ) / 10000

radianceBandX = ((toaBandX * cos(incidenceAngle) * solarIrradianceBandX) / (pi * d2))

where d2 is the earth-sun distance correction. d2 is 1.0/U

The values for incidenceAngle, solarIrradianceBandX and U can be found in the 2 metadata files included in the download.

* In `metadata_msi` we can find the `solarIrradianceBandX`, the `radioAddOffsetBandX` and `U`. See xml content bellow!
* In `metadata_tl` we can find the `incidenceAngle` (which I assume is the solar zenith angle). 

If $J$ is the Julian day of the day of acquisition (day of the year), d2 can be computed as:

d2 = (1-e* cos(0.9856 * (J-4) * pi/180))^2

Where e=0.01673 is the Earth's orbit eccentricity

In [6]:
# This works in the terminal but not in the notebook :/
!gsutil -u cs-starcop-dtacs cat gs://gcp-public-data-sentinel-2/tiles/49/S/GV/S2B_MSIL1C_20220527T030539_N0400_R075_T49SGV_20220527T051042.SAFE/MTD_MSIL1C.xml

ServiceException: 401 Requester pays bucket access requires authentication.


Solar irradiance values, U and radio offsets are in the `s2obj.metadata_msi` file. 


```
[...]
           <QUANTIFICATION_VALUE unit="none">10000</QUANTIFICATION_VALUE>
           <Radiometric_Offset_List>
        <RADIO_ADD_OFFSET band_id="0">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="1">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="2">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="3">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="4">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="5">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="6">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="7">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="8">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="9">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="10">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="11">-1000</RADIO_ADD_OFFSET>
        <RADIO_ADD_OFFSET band_id="12">-1000</RADIO_ADD_OFFSET>
      </Radiometric_Offset_List>
            <Reflectance_Conversion>
        <U>0.975631110815927</U>
        <Solar_Irradiance_List>
          <SOLAR_IRRADIANCE bandId="0" unit="W/m²/µm">1874.3</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="1" unit="W/m²/µm">1959.75</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="2" unit="W/m²/µm">1824.93</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="3" unit="W/m²/µm">1512.79</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="4" unit="W/m²/µm">1425.78</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="5" unit="W/m²/µm">1291.13</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="6" unit="W/m²/µm">1175.57</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="7" unit="W/m²/µm">1041.28</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="8" unit="W/m²/µm">953.93</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="9" unit="W/m²/µm">817.58</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="10" unit="W/m²/µm">365.41</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="11" unit="W/m²/µm">247.08</SOLAR_IRRADIANCE>
          <SOLAR_IRRADIANCE bandId="12" unit="W/m²/µm">87.75</SOLAR_IRRADIANCE>
        </Solar_Irradiance_List>
</Reflectance_Conversion>
[...]
```

In that file there's also the spectral response of each of the bands:

```
[...]
            <Spectral_Information_List>
        <Spectral_Information bandId="0" physicalBand="B1">
          <RESOLUTION>60</RESOLUTION>
          <Wavelength>
            <MIN unit="nm">411</MIN>
            <MAX unit="nm">456</MAX>
            <CENTRAL unit="nm">442.3</CENTRAL>
          </Wavelength>
          <Spectral_Response>
            <STEP unit="nm">1</STEP>
            <VALUES>0.0062411 0.01024045 0.00402983 0.00642179 0.00552753 0.0065525 0.00409887 0.006297 0.00436742 0.00233356 0.00058162 0.00202276 0.00294328 0.00485362 0.00317041 0.00237657 0.00234612 0.00440152 0.01292397 0.05001678 0.18650104 0.45441623 0.72307877 0.83999211 0.86456334 0.87472096 0.89215296 0.91090814 0.92588017 0.93924094 0.94491826 0.95078529 0.96803023 0.99939195 1 0.97548364 0.96148351 0.94986211 0.91841452 0.87989802 0.80383677 0.59752075 0.30474132 0.10798014 0.0304465 0.00885119</VALUES>
          </Spectral_Response>
        </Spectral_Information>
        <Spectral_Information bandId="1" physicalBand="B2">
        
[...]
```